<a href="https://colab.research.google.com/github/duyanh78/O-RDC-Algorithm/blob/main/ORDC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import random
from google.colab import drive
import os
import logging
from geopy.distance import geodesic

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Connect to Google Drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive


In [ ]:
import random

class CacheEntry:
    def __init__(self, request, remaining_time, state="initialized"):
        self.request = request
        self.remaining_time = remaining_time
        self.state = state

class EdgeNode:
    def __init__(self, site_id, latitude, longitude, name, cpu_frequency, hardware_capacity):
        self.site_id = site_id
        self.latitude = latitude
        self.longitude = longitude
        self.name = name
        self.cpu_frequency = cpu_frequency
        self.hardware_capacity = hardware_capacity
        self.available_ram = hardware_capacity  # Initialize available RAM as total hardware capacity
        self.cache = {t: [] for t in range(100)}  # Initialize cache with 50 empty lists
        self.request_counter = 0  # Counter to assign unique IDs to requests

    def __repr__(self):
        return f"{self.site_id}-{self.name}: ({self.latitude}, {self.longitude}), CPU: {self.cpu_frequency} GHz, Capacity: {self.hardware_capacity} MB, Available RAM: {self.available_ram} MB"

    def can_store_container(self, container_size):
        return self.available_ram >= container_size

    def store_request(self, request, remaining_time, current_time_slot):
        self.request_counter += 1  # Increment the counter for each new request
        unique_request_id = self.request_counter  # Assign the counter value as the unique ID
        unique_request = Request(unique_request_id, request.request_type, request.size, request.timestamp)
        if self.can_store_container(request.size):
            self.available_ram -= request.size  # Decrease available RAM
            if self.cache[current_time_slot] is None:
                self.cache[current_time_slot] = []
            self.cache[current_time_slot].append(CacheEntry(unique_request, remaining_time, state="initialized"))
            return True
        return False

    def update_cache(self, current_time_slot):
        destroyed_cont = {k: {t: 0 for t in range(100)} for k in range(1, 51)}
        new_cache = []
        previous_time_slot = current_time_slot - 1
        if previous_time_slot in self.cache and self.cache[previous_time_slot] is not None:
            # Tạo bản sao của các entry trước khi cập nhật remaining_time
            current_entries = [CacheEntry(entry.request, entry.remaining_time, entry.state) for entry in self.cache[previous_time_slot]]
            self.cache[current_time_slot] = current_entries
            for entry in current_entries:
                entry.remaining_time -= 1
                if entry.remaining_time <= 0:  # Nếu container hết thời gian remaining_time
                    self.remove_container(entry, current_time_slot)
                    destroyed_cont[entry.request.request_type][current_time_slot] += 1
                else:
                    if entry.state == "served":
                        entry.state = "stored"  # Chuyển trạng thái sang lưu trữ trong time-slot tiếp theo
                    if entry.state == "initialized":
                        entry.state = "stored"
                    new_cache.append(entry)
        self.cache[current_time_slot] = new_cache if new_cache else None
        return destroyed_cont

    def calculate_a_n_k_t(self, current_time_slot):
        count = {k: {t: 0 for t in range(100)} for k in range(1, 51)}
        if current_time_slot in self.cache:
            if self.cache[current_time_slot] is None:
                for k in range(1, 51):
                    count[k][current_time_slot] = 0
            else:
                for entry in self.cache[current_time_slot]:
                    if entry.remaining_time > 0 and entry.state == "stored" :  # Chỉ tính các container ở trạng thái lưu trữ
                        count[entry.request.request_type][current_time_slot] += 1
        return count

    def remove_container(self, entry, current_time_slot):
        self.available_ram += entry.request.size
        if self.available_ram > self.hardware_capacity:
            self.available_ram = self.hardware_capacity  # Ensure available_ram does not exceed hardware_capacity
        if self.cache[current_time_slot] is not None:
            self.cache[current_time_slot] = [e for e in self.cache[current_time_slot] if e != entry]

    def update_container(self, entry, new_remaining_time, current_time_slot):
        entry.remaining_time = new_remaining_time
        entry.state = "served"  # Chuyển trạng thái sang đã phục vụ sau khi phục vụ
        # Lưu lại entry vào cache
        self.remove_container(entry, current_time_slot)
        if self.cache[current_time_slot] is None:
            self.cache[current_time_slot] = []
        self.cache[current_time_slot].append(entry)


from geopy.distance import geodesic

class Zone:
    def __init__(self, zone_id, nodes):
        self.zone_id = zone_id
        self.nodes = nodes
        self.instantiation_costs = {}
        self.retention_costs = {}

    def calculate_costs(self, ranked_functions):
        for node in self.nodes:
            for k, (hash_function, invocations, size) in ranked_functions.items():
                dnk = size / node.cpu_frequency  # Instantiation cost
                rnk = size * node.cpu_frequency  # Retention cost
                self.instantiation_costs[(node.site_id, k)] = dnk
                self.retention_costs[(node.site_id, k)] = rnk

    def get_zone_center(self):
        latitudes = [node.latitude for node in self.nodes]
        longitudes = [node.longitude for node in self.nodes]
        center_latitude = sum(latitudes) / len(latitudes)
        center_longitude = sum(longitudes) / len(longitudes)
        return (center_latitude, center_longitude)




In [ ]:
def load_edge_nodes(file_path, zone_node_ids):
    cpu_frequencies = [2.4, 2.7, 3.0, 3.3, 3.6]
    hardware_capacities = np.arange(16000, 24000, 1000)

    df = pd.read_csv(file_path)
    edge_nodes = []

    for zone_id, node_ids in zone_node_ids.items():
        assigned_frequencies = []
        zone_nodes = []
        for index, node_id in enumerate(node_ids):
            row = df[df['SITE_ID'] == node_id].iloc[0]
            if len(assigned_frequencies) < 5:
                cpu_frequency = cpu_frequencies[len(assigned_frequencies)]
                assigned_frequencies.append(cpu_frequency)
            else:
                cpu_frequency = random.choice(cpu_frequencies)
            edge_node = EdgeNode(row.SITE_ID, row.LATITUDE, row.LONGITUDE, row.NAME, cpu_frequency, random.choice(hardware_capacities))
            zone_nodes.append(edge_node)

        # Ensure each zone has all 5 cpu frequencies
        missing_frequencies = set(cpu_frequencies) - set(assigned_frequencies)
        if missing_frequencies:
            for missing_freq in missing_frequencies:
                index = random.choice(range(len(zone_nodes)))
                zone_nodes[index].cpu_frequency = missing_freq

        edge_nodes.extend(zone_nodes)

    return edge_nodes

file_path = "eua-dataset-master/eua-dataset-master/edge-servers/site-optus-melbCBD.csv"
zone_node_ids = {
    1: [304365, 9026103, 304366, 135011, 101636, 135045, 42987, 302516],
    2: [101373, 9009845, 130005, 134901, 134680, 303255, 9013096],
    3: [134574, 134906, 302517, 134454, 49873, 301895, 50151, 134990, 301896],
    4: [101381, 461423, 304060, 302923, 302854, 9009843, 49630, 135213],
    5: [134317, 301393, 10003238, 304368, 50226, 47316, 302571, 50686, 134733, 304369],
    6: [135237, 9002262, 135073, 305394, 304744, 10003026, 134453, 51576, 11590, 134360],
    7: [134754, 130439, 301240, 134565, 301361, 301383, 135330],
    8: [134329, 135253, 51718, 303712, 51622, 304434, 301382],
    9: [301388, 44125, 303676, 10004167, 301386, 134386, 301645, 9014605, 9014989],
    10: [134403, 301208, 44101, 134554, 11600, 9015396, 306249, 134980, 11601, 301658],
    11: [135009, 101385, 11571, 41660, 303652, 50669, 134941, 134822],
    12: [51590, 206082, 9014611, 304562, 34603, 304364, 303710, 134449],
    13: [135390, 135143, 404118, 10003027, 11593, 11591, 9001289],
    14: [304370, 135231, 135306, 11579, 10004576, 301205, 11581],
    15: [134923, 53003, 134245, 11599, 304371, 134547, 9009844, 134872, 304363, 134857]
}

all_nodes = load_edge_nodes(file_path, zone_node_ids)

zones = {zone_id: Zone(zone_id, [node for node_id in node_ids for node in all_nodes if node.site_id == node_id])
         for zone_id, node_ids in zone_node_ids.items()}

# Sort nodes in each zone by CPU frequency and reassign site_id
for zone_id, zone in zones.items():
    sorted_nodes = sorted(zone.nodes, key=lambda node: node.cpu_frequency)
    for idx, node in enumerate(sorted_nodes):
        node.site_id = idx + 1
    zone.nodes = sorted_nodes
# Display sorted zones
for zone_id, zone in zones.items():
    print(f"Zone {zone_id}:")
    for node in zone.nodes:
        print(node)
    print()



Zone 1:
1-Optus Site 318-326 Spencer St MELBOURNE: (-37.812934000000006, 144.952075), CPU: 2.4 GHz, Capacity: 19000 MB, Available RAM: 19000 MB
2-Vodafone Site Building C William Angliss Institute 555 La Trobe Street MELBOURNE VIC 3000: (-37.813175, 144.952919), CPU: 2.7 GHz, Capacity: 19000 MB, Available RAM: 19000 MB
3-Optus Site Flagstaff Station 433 Latrobe Street MELBOURNE: (-37.811928, 144.956414), CPU: 2.7 GHz, Capacity: 17000 MB, Available RAM: 17000 MB
4-Optus Site 450 LaTrobe St MELBOURNE: (-37.812343, 144.954135), CPU: 3.0 GHz, Capacity: 20000 MB, Available RAM: 20000 MB
5-Upper Flagstaff Station MELBOURNE: (-37.812379, 144.956402), CPU: 3.0 GHz, Capacity: 20000 MB, Available RAM: 20000 MB
6-Optus Site CGU Centre 485 Latrobe Street MELBOURNE: (-37.812799, 144.954686), CPU: 3.3 GHz, Capacity: 19000 MB, Available RAM: 19000 MB
7-Nubrick House 271 William Street MELBOURNE: (-37.812653000000005, 144.956622), CPU: 3.3 GHz, Capacity: 21000 MB, Available RAM: 21000 MB
8-CGU Bldg 48

In [ ]:
class User:
    def __init__(self, user_id, latitude, longitude):
        self.user_id = user_id
        self.latitude = latitude
        self.longitude = longitude

    def __repr__(self):
        return f"User({self.user_id}): ({self.latitude}, {self.longitude})"


class Request:
    def __init__(self, user_id, request_type, size, timestamp):
        self.user_id = user_id
        self.request_type = request_type
        self.size = size
        self.timestamp = timestamp

    def __repr__(self):
        return f"Request(User: {self.user_id}, Type: {self.request_type}, Size: {self.size}MB, Timestamp: {self.timestamp})"


In [ ]:
def load_users(file_path):
    df = pd.read_csv(file_path, delimiter=',')
    users = []
    for index, row in df.iterrows():
        user_id = index + 1
        latitude = row['Latitude']
        longitude = row['Longitude']
        user = User(user_id, latitude, longitude)
        users.append(user)
    logging.info(f"Loaded {len(users)} users from {file_path}")
    return users



def load_invocation_data(file_path):
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        logging.info(f"Loaded invocation data from {file_path}")
        return df
    else:
        logging.error(f"File not found: {file_path}")
        return None

def load_duration_data(file_path):
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        logging.info(f"Loaded duration data from {file_path}")
        return df
    else:
        logging.error(f"File not found: {file_path}")
        return None

def filter_functions_by_hashowner(data, hash_owner):
    filtered_data = data[data['HashOwner'] == hash_owner]
    logging.info(f"Filtered functions by hash owner {hash_owner}, resulting in {len(filtered_data)} entries")
    return filtered_data

def generate_requests_zipf(beta, num_requests, K):
    ranks = np.arange(1, K + 1)
    weights = 1 / np.power(ranks, beta)
    weights /= weights.sum()
    return np.random.choice(ranks, size=num_requests, p=weights)

def generate_user_requests(users, beta, ranked_functions, num_requests_per_timeslot):
    K = len(ranked_functions)
    total_time_slots = 100  # Assume 1 day, each day has 1440 minutes
    requests = []

    for timestamp in range(total_time_slots):
        for user in users:
            user_id = user.user_id
            # Generate request types randomly according to Zipf-β distribution
            request_types = generate_requests_zipf(beta, num_requests_per_timeslot, K)
            for request_type_rank in request_types:
                function_info = ranked_functions[request_type_rank]
                request_type = function_info[0]
                size = function_info[2]
                requests.append(Request(user_id, request_type_rank, size, timestamp))
                logging.debug(f"Generated request: {requests[-1]}")
    logging.info(f"Generated a total of {len(requests)} requests")
    return requests



user_file_path = "eua-dataset-master/eua-dataset-master/users/users-melbcbd-generated.csv"
users = load_users(user_file_path)

extracted_folder = "extracted_files/azure-functions-master"
invocation_file_paths = [os.path.join(extracted_folder, "invocations_per_function_md.anon.d01.csv")]
duration_file_paths = [os.path.join(extracted_folder, "function_durations_percentiles.anon.d01.csv")]

invocation_data = [load_invocation_data(file) for file in invocation_file_paths if os.path.exists(file)]
duration_data = [load_duration_data(file) for file in duration_file_paths if os.path.exists(file)]

invocation_data = [df for df in invocation_data if df is not None]
duration_data = [df for df in duration_data if df is not None]

if invocation_data:
    all_invocations = pd.concat(invocation_data, ignore_index=True)
    logging.info("Combined all invocation data into a single DataFrame")

if duration_data:
    all_durations = pd.concat(duration_data, ignore_index=True)
    logging.info("Combined all duration data into a single DataFrame")

selected_hash_owner = '98f3fe1d01789fc2d8ab1dc9cd9a31d51fcbf1936e36a47526280c95ec0cd944'
filtered_functions = filter_functions_by_hashowner(all_invocations, selected_hash_owner)
filtered_durations = filter_functions_by_hashowner(all_durations, selected_hash_owner)

filtered_functions['TotalInvocations'] = filtered_functions.loc[:, '1':'1440'].sum(axis=1)
request_counts = filtered_functions.groupby('HashFunction')['TotalInvocations'].sum().sort_values(ascending=False)
selected_functions = request_counts.head(50)
logging.info(f"Selected top 50 functions based on total invocations")

function_sizes = filtered_durations.groupby('HashFunction')['Average'].mean()
function_sizes = function_sizes[selected_functions.index]

min_size = 20  # MB
max_size = 250  # MB
sizes = np.interp(function_sizes, (function_sizes.min(), function_sizes.max()), (min_size, max_size))

ranked_functions_list = [(i + 1, selected_functions.index[i], selected_functions.iloc[i], sizes[i]) for i in range(50)]
logging.info("Ranked functions and assigned sizes based on average execution time")

ranked_functions = {rank: (hash_function, invocations, size) for rank, hash_function, invocations, size in ranked_functions_list}

print("Ranked Functions:")
for rank, (hash_function, invocations, size) in ranked_functions.items():
    print(f"Rank {rank}: Function {hash_function}, Invocations: {invocations}, Estimated Size: {size:.2f}MB")

# beta = 0.5  # Adjust this value as needed
# num_requests_per_timeslot =   # Assume each user generates 2 requests per timeslot
# requests = generate_user_requests(users, beta, ranked_functions, num_requests_per_timeslot)

# print("Generated Requests:")
# for request in requests[:10]:  # Display the first 10 requests
#     print(request)


Ranked Functions:
Rank 1: Function 85cff28b83792428e039e986e5d5554318a9678f111ec9a6f65ae2956f25ed18, Invocations: 1985730, Estimated Size: 21.31MB
Rank 2: Function 6c871093253858350d730f80cfbc5109aa2529d9cda37341989ea8854485663e, Invocations: 1984364, Estimated Size: 20.73MB
Rank 3: Function 2f5fefaa527316e6630e589235511e7f13781841babdc4c15ecc33a28204c9b4, Invocations: 22307, Estimated Size: 27.79MB
Rank 4: Function 331a2f8a0962ecde7e31c20b068efc25c79deb733b3bfe4094dffcb2d7be4a9f, Invocations: 17228, Estimated Size: 20.12MB
Rank 5: Function 2a48e6ede657e3a9cbc0d374df1bb55c25620bca2787e98e26af975927981f13, Invocations: 11482, Estimated Size: 22.65MB
Rank 6: Function 4cd30dc5400766df9735dd7616dda2ef599e4ca10b3c4cbceaca08c0ff6a7baa, Invocations: 7288, Estimated Size: 21.06MB
Rank 7: Function 94f7fff69733d5538b07436d65e1b73cac99d26705b43b3d623892302123fdde, Invocations: 3470, Estimated Size: 37.64MB
Rank 8: Function b937c1821c6ede34645cbf2d9f6cc8bcd679c7398da078bcb23357beeb05af88, Invocati

<ipython-input-5-84c815d4db32>:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_functions['TotalInvocations'] = filtered_functions.loc[:, '1':'1440'].sum(axis=1)


In [ ]:
beta = 1  # Adjust this value as needed
num_requests_per_timeslot= 20
requests = generate_user_requests(users, beta, ranked_functions, num_requests_per_timeslot)

In [ ]:
def calculate_lambda_n_k_t(users, requests, zones, K, total_time_slots):
    # Initialize dictionary to store the total request load λ_{n,k,t}
    lambda_n_k_t = {}

    # Function to find the nearest node in a given zone for a user
    def find_nearest_node(user, nodes):
        min_distance = float('inf')
        nearest_node = None
        for node in nodes:
            distance = geodesic((user.latitude, user.longitude), (node.latitude, node.longitude)).kilometers
            if distance < min_distance:
                min_distance = distance
                nearest_node = node
        return nearest_node

    # Determine the nearest node for each user within their assigned zone
    user_assignments = {}  # Dictionary to store nearest node for each user
    for user in users:
        nearest_node = None
        nearest_zone = None
        min_distance = float('inf')

        for zone_id, zone in zones.items():
            node = find_nearest_node(user, zone.nodes)
            if node:
                distance = geodesic((user.latitude, user.longitude), (node.latitude, node.longitude)).kilometers
                if distance < min_distance:
                    min_distance = distance
                    nearest_node = node
                    nearest_zone = zone_id

        user_assignments[user.user_id] = (nearest_zone, nearest_node.site_id)

    # Assign requests to the nearest node in their zone and calculate λ_{n,k,t}
    for request in requests:
        user_id = request.user_id
        k = request.request_type  # Container type rank
        t = request.timestamp  # Time slot, assuming 1 time slot = 1 minute

        nearest_zone, nearest_node_id = user_assignments[user_id]
        n = nearest_node_id

        if (nearest_zone, n, k, t) not in lambda_n_k_t:
            lambda_n_k_t[(nearest_zone, n, k, t)] = 0

        lambda_n_k_t[(nearest_zone, n, k, t)] += 1  # Incrementing the count of requests for the nearest node

    return lambda_n_k_t


In [ ]:
total_time_slots=100
K = 50
lambda_n_k_t =calculate_lambda_n_k_t(users, requests, zones, K, total_time_slots)

In [ ]:
from collections import OrderedDict

def sort_lambda_n_k_t(lambda_n_k_t):
    # Sắp xếp từ điển theo thứ tự (t, zone_id, node_id, k)
    sorted_lambda_n_k_t = OrderedDict(sorted(lambda_n_k_t.items(), key=lambda x: (x[0][3], x[0][0], x[0][1], x[0][2])))
    return sorted_lambda_n_k_t

def print_zone_1_sorted_lambda_n_k_t(sorted_lambda_n_k_t):
    # In các giá trị lambda(n,k,t) trong zone 1
    for (zone_id, node_id, k, t), count in sorted_lambda_n_k_t.items():
        if zone_id == 1:
            print(f"zone {zone_id}: lambda({node_id}, {k}, {t}) = {count}")

# Giả sử lambda_n_k_t là dictionary chứa số request loại k gửi đến node n tại time-slot t
# lambda_n_k_t = {(zone_id, node_id, k, t): count, ...}

sorted_lambda_n_k_t = sort_lambda_n_k_t(lambda_n_k_t)
print_zone_1_sorted_lambda_n_k_t(sorted_lambda_n_k_t)


Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
zone 1: lambda(5, 12, 76) = 1
zone 1: lambda(5, 17, 76) = 1
zone 1: lambda(5, 18, 76) = 1
zone 1: lambda(5, 19, 76) = 1
zone 1: lambda(5, 24, 76) = 1
zone 1: lambda(5, 28, 76) = 1
zone 1: lambda(5, 34, 76) = 1
zone 1: lambda(5, 35, 76) = 1
zone 1: lambda(5, 39, 76) = 1
zone 1: lambda(6, 1, 76) = 10
zone 1: lambda(6, 2, 76) = 10
zone 1: lambda(6, 3, 76) = 2
zone 1: lambda(6, 4, 76) = 3
zone 1: lambda(6, 5, 76) = 1
zone 1: lambda(6, 6, 76) = 4
zone 1: lambda(6, 7, 76) = 5
zone 1: lambda(6, 8, 76) = 3
zone 1: lambda(6, 9, 76) = 1
zone 1: lambda(6, 10, 76) = 1
zone 1: lambda(6, 11, 76) = 4
zone 1: lambda(6, 12, 76) = 2
zone 1: lambda(6, 13, 76) = 5
zone 1: lambda(6, 14, 76) = 2
zone 1: lambda(6, 15, 76) = 2
zone 1: lambda(6, 16, 76) = 1
zone 1: lambda(6, 17, 76) = 1
zone 1: lambda(6, 18, 76) = 2
zone 1: lambda(6, 19, 76) = 1
zone 1: lambda(6, 20, 76) = 2
zone 1: lambda(6, 22, 76) = 1
zone 1: lambda(6, 28, 76) = 2
zone 1: lambda(6, 30

In [ ]:
# Calculate instantiation and retention costs for all zones
for zone in zones.values():
    zone.calculate_costs(ranked_functions)

# Function to calculate communication costs
def calculate_communication_cost(zones, K):
    communication_costs = {}
    max_distance = 0
    for zone_id_1, zone_1 in zones.items():
        for zone_id_2, zone_2 in zones.items():
            if zone_id_1 != zone_id_2:
                # Calculate geographical distance between zone centers
                center_1 = zone_1.get_zone_center()
                center_2 = zone_2.get_zone_center()
                distance = geodesic(center_1, center_2).kilometers
                if max_distance < distance:
                    max_distance = distance
                for k in range(1, K + 1):
                    try:
                        # Calculate minimal and maximal instantiation costs for container type k in both zones
                        min_inst_cost_1 = min(zone_1.instantiation_costs[(node.site_id, k)] for node in zone_1.nodes)
                        max_inst_cost_1 = max(zone_1.instantiation_costs[(node.site_id, k)] for node in zone_1.nodes)
                        min_inst_cost_2 = min(zone_2.instantiation_costs[(node.site_id, k)] for node in zone_2.nodes)
                        max_inst_cost_2 = max(zone_2.instantiation_costs[(node.site_id, k)] for node in zone_2.nodes)

                        # Calculate average min and max costs
                        avg_min_cost = (min_inst_cost_1 + min_inst_cost_2) / 2
                        avg_max_cost = (max_inst_cost_1 + max_inst_cost_2) / 2

                        # Normalize distance to be between 0 and 1
                        normalized_distance = distance / max_distance

                        # Calculate communication cost
                        comm_cost = avg_min_cost + (avg_max_cost - avg_min_cost) * normalized_distance

                        communication_costs[(zone_id_1, zone_id_2, k)] = comm_cost
                    except KeyError:
                        print(f"Missing instantiation cost for container type {k} in zones {zone_id_1} or {zone_id_2}")

    return communication_costs

# Calculate communication costs for all pairs of zones and all container types
K = 50  # Assuming there are 50 types of containers
communication_costs = calculate_communication_cost(zones, K)

# Print the communication costs for each container type
for (zone_id_1, zone_id_2, k), cost in communication_costs.items():
    print(f"Communication cost for container type {k} between Zone {zone_id_1} and Zone {zone_id_2}: {cost:.2f}")


Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
Communication cost for container type 1 between Zone 8 and Zone 14: 7.30
Communication cost for container type 2 between Zone 8 and Zone 14: 7.11
Communication cost for container type 3 between Zone 8 and Zone 14: 9.52
Communication cost for container type 4 between Zone 8 and Zone 14: 6.90
Communication cost for container type 5 between Zone 8 and Zone 14: 7.76
Communication cost for container type 6 between Zone 8 and Zone 14: 7.22
Communication cost for container type 7 between Zone 8 and Zone 14: 12.90
Communication cost for container type 8 between Zone 8 and Zone 14: 7.66
Communication cost for container type 9 between Zone 8 and Zone 14: 7.06
Communication cost for container type 10 between Zone 8 and Zone 14: 15.50
Communication cost for container type 11 between Zone 8 and Zone 14: 6.86
Communication cost for container type 12 between Zone 8 and Zone 14: 15.80
Communication cost for container type 13 between Zone 8 and Z

In [ ]:
import numpy as np

def calculate_D(phi, D, theta, b, n, d, N):
    if n < N:
        exp_term1 = np.exp(b[n+1] / theta[n]) - 1
        exp_term2 = np.exp(b[n+1] / theta[n+1]) - 1
        phi_ratio = (d[n] / d[n+1]) * np.exp((b[n+1] / theta[n]) - (b[n+1] / theta[n+1]))
        return phi_ratio * D[n+1] - theta[n] * exp_term1 + phi_ratio * theta[n+1] * exp_term2
    else:
        return D[n]  # For the last element

def calculate_b(d, r, D, theta, alpha, n, N, b):
    if n == 1:
        return d[N] / (alpha * r[1])  # Special case for b[1]
    elif n > 1 and d[n] != 0 and r[n] != 0 and r[n-1] != 0:
        if r[n] == r[n-1]:
            return b[n+1]
        if (d[n-1] * r[n] - d[n] * r[n-1]) * (1 - D[n] * alpha * r[n] / d[n]) > 0:
            try:
                return theta[n] * np.log(((d[n-1] * r[n] - d[n] * r[n-1]) * (1 - D[n] * alpha * r[n] / d[n])) / (d[n] * (r[n] - r[n-1])))
            except ValueError:
                return b[n]
        else:
            return b[n]
    else:
        return b[n]

def calculate_phi(phi, d, b, theta, n):
    if n == 1:
        return 1  # phi[1] is always 1
    elif n > 1 and d[n] != 0 and d[n-1] != 0 and theta[n-1] != 0 and theta[n] != 0:
        return phi[n-1] * (d[n-1] / d[n]) * np.exp(b[n] / theta[n-1] - b[n] / theta[n])
    else:
        return phi[n]

def calculate_omega(phi, theta, b, n):
    if n > 0 and theta[n] != 0:
        return (phi[n] * theta[n]) * (np.exp(b[n] / theta[n]) - np.exp(b[n+1] / theta[n]))
    else:
        return 0

def calculate_p_n(x, phi, theta, b, n):
    if b[n+1] <= x <= b[n]:
        return phi[n] * np.exp(x / theta[n])
    else:
        return 0

def dist_algorithm(zone_nodes, instantiation_costs, retention_costs, alpha, k):
    N = len(zone_nodes)  # Number of nodes in the zone
    d = np.zeros(N+1)
    r = np.zeros(N+1)

    for node in zone_nodes:
        d[node.site_id] = instantiation_costs[(node.site_id, k)]
        r[node.site_id] = retention_costs[(node.site_id, k)]

    theta = np.zeros(N+1)
    for i in range(1, N+1):
        theta[i] = d[i] / (alpha * r[i])

    print(f"Zone Nodes: {zone_nodes}")
    print(f"Container Type {k}")
    print("d:", d)
    print("r:", r)

    phi = np.zeros(N+1)
    b = np.zeros(N+2)
    D = np.zeros(N+1)
    omega = np.zeros(N+1)
    x = np.zeros(N+1)
    D[N] = 0  # Initialize D[N]
    b[N+1] = 0  # Initialize b[N+1]

    for n in range(N, 0, -1):  # Compute b and D, correct loop range for decreasing n
        b[n] = calculate_b(d, r, D, theta, alpha, n, N, b)
        if n < N:
            D[n] = calculate_D(phi, D, theta, b, n, d, N)

    for n in range(1, N+1):  # Compute phi
        phi[n] = calculate_phi(phi, d, b, theta, n)
    for n in range(1, N+1):  # Compute x
        x[n] = (b[n+1] + b[n]) / 2
    p_n_values = [calculate_p_n(0.5 * (b[n+1] + b[n]), phi, theta, b, n) for n in range(1, N+1)]
    omega_sum = sum([calculate_omega(phi, theta, b, n) for n in range(1, N+1)])  # sum of omega from 1 to N
    if omega_sum != 0:
        for n in range(1, N+1):
            phi[n] = phi[n] / omega_sum

    for n in range(1, N+1):  # Compute omega
        omega[n] = calculate_omega(phi, theta, b, n)


    return b, x[1:], omega[1:], p_n_values  # Exclude the first element as it is zero

# Hàm tổng quát để tính toán DIST cho mọi loại k của mọi zone
def calculate_dist_for_all_zones_and_k(zones, alpha, K):
    results = {}

    for zone_id, zone in zones.items():
        results[zone_id] = {}
        for k in range(1, K + 1):
            b, x, omega, p_n_values = dist_algorithm(zone.nodes, zone.instantiation_costs, zone.retention_costs, alpha, k)
            results[zone_id][k] = {
                'b': b,
                'x': x,
                'omega': omega,
                'p_n_values': p_n_values
            }
            print(f"Zone {zone_id}, Container Type {k}")
            print("b:", b)
            print("x:", x)
            print("omega:", omega)
            print("p_n_values:", p_n_values)
            print()
    return results

# Ví dụ sử dụng hàm tổng quát
alpha = 0.04  # Giá trị ví dụ cho alpha
K = 50  # Số lượng loại container

# Giả sử instantiation_costs và retention_costs đã được tính trước đó
dist_results = calculate_dist_for_all_zones_and_k(zones, alpha, K)


Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
 0.14967171 0.1090705  0.40150104]
p_n_values: [1.8797141631326963, 1.8140697141424904, 1.894759181268279, 1.7591446223614917, 1.7591446223614917, 1.7591446223614917, 1.8278980045366964, 1.7689402913663594, 1.2554799067671185]

Zone Nodes: [1-Telstra/Optus Site cnr Russell & Collins Sts MELBOURNE: (-37.814765, 144.969286), CPU: 2.4 GHz, Capacity: 17000 MB, Available RAM: 17000 MB, 2-Optus Site Collins & Swanston Cnr Collins & Swanston Sts MELBOURNE: (-37.815549, 144.966686), CPU: 2.4 GHz, Capacity: 17000 MB, Available RAM: 17000 MB, 3-Hyatt Hotel 123 Collins St MELBOURNE: (-37.815309, 144.969476), CPU: 2.7 GHz, Capacity: 19000 MB, Available RAM: 19000 MB, 4-Optus/Vodafone Site 55 Swanston Walk MELBOURNE: (-37.816206, 144.966634), CPU: 2.7 GHz, Capacity: 18000 MB, Available RAM: 18000 MB, 5-Customer Site 271 Collins Street MELBOURNE: (-37.816496, 144.96509), CPU: 2.7 GHz, Capacity: 16000 MB, Available RAM: 16000 MB, 6-Optus Roofto

In [ ]:
def allocate_requests(num_requests, omega):
    total_requests = int(num_requests)
    allocation = np.floor(np.array(omega) * total_requests).astype(int)
    remaining_requests = total_requests - allocation.sum()

    if remaining_requests > 0:
        fractional_parts = np.array(omega) * total_requests - allocation
        indices = np.argsort(-fractional_parts)
        for i in indices[:remaining_requests]:
            allocation[i] += 1

    return allocation.tolist()


In [ ]:
import numpy as np
import random

def ORDC_algorithm(zones, K, total_time_slots, dist_results, lambda_n_k_t, communication_costs):
    # Khởi tạo các biến lưu trữ kết quả
    a_n_k_t = {zone_id: {node.site_id: {k: [0] * total_time_slots for k in range(1, K + 1)} for node in zone.nodes} for zone_id, zone in zones.items()}
    y_n_k_t = {zone_id: {node.site_id: {k: [0] * total_time_slots for k in range(1, K + 1)} for node in zone.nodes} for zone_id, zone in zones.items()}
    m_n_k_t = {zone_id: {node.site_id: {k: [0] * total_time_slots for k in range(1, K + 1)} for node in zone.nodes} for zone_id, zone in zones.items()}

    # Khởi tạo mảng chi phí
    CL = np.zeros(total_time_slots)  # Chi phí truyền thông
    CR_I = np.zeros(total_time_slots)  # Chi phí lưu trữ và khởi tạo
    CR_R = np.zeros(total_time_slots)  # Chi phí lưu trữ và khởi tạo

    for t in range(total_time_slots):
        print(f"\n--- Time-slot {t} ---")
        # Cập nhật cache cho tất cả các node vào đầu mỗi time-slot
        for z, zone in zones.items():
            for node in zone.nodes:
                y_n_k_t[z][node.site_id] = node.update_cache(current_time_slot=t)

        for z, zone in zones.items():
            for k in range(1, K + 1):
                lambda_k_t = sum(lambda_n_k_t.get((z, node.site_id, k, t), 0) for node in zone.nodes)
                a_k_t = sum(node.calculate_a_n_k_t(current_time_slot=t)[k][t] for node in zone.nodes)

                if lambda_k_t <= a_k_t:
                    remaining_lambda_k_t = lambda_k_t
                    allocations = allocate_requests(lambda_k_t, [dist_results[z][k]['omega'][node.site_id - 1] for node in zone.nodes])

                    for i, node in enumerate(zone.nodes):
                        m_n_k_t[z][node.site_id][k][t] = allocations[i]
                        remaining_lambda_k_t -= m_n_k_t[z][node.site_id][k][t]

                        for _ in range(m_n_k_t[z][node.site_id][k][t]):
                            if node.cache[t] is not None:
                                entry = next((entry for entry in node.cache[t] if entry.request.request_type == k and entry.state == "stored"), None)
                            else:
                                entry = None

                            if entry:
                                entry.state = "served"
                                CR_R[t] += alpha * zone.retention_costs[(node.site_id, k)]
                                if random.random() > dist_results[z][k]['p_n_values'][node.site_id - 1]:
                                    node.remove_container(entry, t)
                                else:
                                    node.update_container(entry, dist_results[z][k]['x'][node.site_id - 1], current_time_slot=t)
                            else:
                                if node.can_store_container(ranked_functions[k][2]):
                                    remaining_time = dist_results[z][k]['x'][node.site_id - 1]
                                    request = Request(user_id=1, request_type=k, size=ranked_functions[k][2], timestamp=t)
                                    node.store_request(request, remaining_time, current_time_slot=t)
                                    CR_I[t] += zone.instantiation_costs[(node.site_id, k)]
                                else:
                                    print(f"Node {node.site_id} cannot store container type {k}")

                else:
                    excess_requests = lambda_k_t - a_k_t

                    for node in zone.nodes:
                        for other_zone_id, other_zone in zones.items():
                            if other_zone_id != z:
                                for other_node in other_zone.nodes:
                                    if communication_costs[(z, other_zone_id, k)] < zone.instantiation_costs[(node.site_id, k)]:
                                        a_n_k_t[other_zone_id][other_node.site_id][k][t] = other_node.calculate_a_n_k_t(current_time_slot=t)[k][t]
                                        if a_n_k_t[other_zone_id][other_node.site_id][k][t] > 0:
                                            allocated_requests = min(excess_requests, a_n_k_t[other_zone_id][other_node.site_id][k][t])
                                            excess_requests -= allocated_requests
                                            a_n_k_t[other_zone_id][other_node.site_id][k][t] -= allocated_requests

                                            for _ in range(allocated_requests):
                                                if other_node.cache[t] is not None:
                                                    entry = next((entry for entry in other_node.cache[t] if entry.request.request_type == k and entry.state == "stored"), None)
                                                else:
                                                    entry = None

                                                if entry:
                                                    CL[t] += communication_costs[(z, other_zone_id, k)]
                                                    entry.state = "served"
                                                    CR_R[t] += alpha * other_zone.retention_costs[(other_node.site_id, k)]
                                                    if random.random() > dist_results[other_zone_id][k]['p_n_values'][other_node.site_id - 1]:
                                                        other_node.remove_container(entry, t)
                                                    else:
                                                        other_node.update_container(entry, dist_results[other_zone_id][k]['x'][other_node.site_id - 1], current_time_slot=t)

                                            if excess_requests <= 0:
                                                break
                            if excess_requests <= 0:
                                break

                    if excess_requests > 0:
                        while excess_requests > 0:
                            node = random.choice(zone.nodes)
                            if node.can_store_container(ranked_functions[k][2]):
                                remaining_requests_to_allocate = min(excess_requests, 1)
                                remaining_time = dist_results[z][k]['x'][node.site_id - 1]
                                request = Request(user_id=1, request_type=k, size=ranked_functions[k][2], timestamp=t)
                                node.store_request(request, remaining_time, current_time_slot=t)
                                excess_requests -= remaining_requests_to_allocate
                                CR_I[t] += zone.instantiation_costs[(node.site_id, k)]

                    remaining_lambda_k_t = a_k_t
                    allocations = allocate_requests(remaining_lambda_k_t, [dist_results[z][k]['omega'][node.site_id - 1] for node in zone.nodes])

                    for i, node in enumerate(zone.nodes):
                        m_n_k_t[z][node.site_id][k][t] = allocations[i]

                        for _ in range(m_n_k_t[z][node.site_id][k][t]):
                            if node.cache[t] is not None:
                                entry = next((entry for entry in node.cache[t] if entry.request.request_type == k and entry.state == "stored"), None)
                            else:
                                entry = None

                            if entry:
                                entry.state = "served"
                                CR_R[t] += alpha * zone.retention_costs[(node.site_id, k)]
                                if random.random() > dist_results[z][k]['p_n_values'][node.site_id - 1]:
                                    node.remove_container(entry, t)
                                else:
                                    node.update_container(entry, dist_results[z][k]['x'][node.site_id - 1], current_time_slot=t)
                            else:
                                if node.can_store_container(ranked_functions[k][2]):
                                    remaining_time = dist_results[z][k]['x'][node.site_id - 1]
                                    request = Request(user_id=1, request_type=k, size=ranked_functions[k][2], timestamp=t)
                                    node.store_request(request, remaining_time, current_time_slot=t)
                                    CR_I[t] += zone.instantiation_costs[(node.site_id, k)]
                                else:
                                    print(f"Node {node.site_id} cannot store container type {k}")

        for z, zone in zones.items():
            for node in zone.nodes:
                if node.cache[t] is not None:
                    for entry in node.cache[t]:
                        if entry.state == "stored":
                            CR_R[t] += alpha * zone.retention_costs[(node.site_id, entry.request.request_type)]

        print(f"Chi phí truyền thông tại time-slot {t}: {CL[t]}")
        print(f"Chi phí lưu trữ tại time-slot {t}: {CR_R[t]}")
        print(f"Chi phí khởi tạo tại time-slot {t}: {CR_I[t]}")

    total_CL = np.sum(CL)
    total_CR_I = np.sum(CR_I)
    total_CR_R = np.sum(CR_R)
    print(f"Tổng chi phí truyền thông: {total_CL}")
    print(f"Tổng chi phí lưu trữ: {total_CR_R}")
    print(f"Tổng chi phí khởi tạo: {total_CR_I}")
    return CL, CR_R, CR_I

# Gọi hàm ORDC_algorithm với các tham số đã chuẩn bị
# CL, CR_R, CR_I = ORDC_algorithm(zones, 50, total_time_slots, dist_results, lambda_n_k_t, communication_costs)


In [ ]:
import sys

# Redirect print statements to a file
class Logger(object):
    def __init__(self, filename="Default.log"):
        self.terminal = sys.stdout
        self.log = open(filename, "w")

    def write(self, message):
        self.terminal.write(message)
        self.log.write(message)

    def flush(self):
        pass

sys.stdout = Logger("output_ORDC_b=1_a=0.04_20xuser.txt")

# Thực hiện tính toán ORDC và in kết quả
ORDC_algorithm(zones, K, total_time_slots, dist_results, lambda_n_k_t, communication_costs)


# Đặt lại stdout về chế độ ban đầu
sys.stdout = sys.__stdout__

print("Kết quả đã được lưu vào file output.txt")



--- Time-slot 0 ---
Chi phí truyền thông tại time-slot 0: 0.0
Chi phí lưu trữ tại time-slot 0: 0.0
Chi phí khởi tạo tại time-slot 0: 192059.29580635898

--- Time-slot 1 ---
Node 7 cannot store container type 38
Node 7 cannot store container type 38
Node 7 cannot store container type 39
Node 7 cannot store container type 39
Node 7 cannot store container type 39
Node 7 cannot store container type 40
Node 7 cannot store container type 40
Node 7 cannot store container type 42
Node 7 cannot store container type 42
Node 7 cannot store container type 42
Node 7 cannot store container type 43
Node 7 cannot store container type 43
Node 7 cannot store container type 43
Node 7 cannot store container type 44
Node 7 cannot store container type 44
Node 7 cannot store container type 45
Node 7 cannot store container type 45
Node 7 cannot store container type 45
Node 7 cannot store container type 46
Node 7 cannot store container type 46
Node 7 cannot store container type 47
Node 7 cannot store containe